### 第1セル

In [12]:
import gradio as gr
import socket

def find_available_port(start_port=7860, max_attempts=100):
    """利用可能なポートを見つける"""
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('0.0.0.0', port))
                return port
        except OSError:
            continue
    return None

class DictionaryChatbot:
    def __init__(self):
        # ルールを定義（キーワード：応答）
        self.rules = {
            "こんにちは": "こんにちは！何かお手伝いできますか？",
            "お疲れ様": "お疲れ様です！",
            "ありがとう": "どういたしまして！",
            "さようなら": "さようなら！またお話ししましょう！"
        }
    
    def find_response(self, user_input):
        """ユーザーの入力から適切な応答を見つける"""
        if not user_input.strip():
            return "何か入力してください。"
        
        user_input = user_input.strip()
        
        # 完全一致をチェック
        if user_input in self.rules:
            return self.rules[user_input]
        
        # デフォルト応答
        return "申し訳ございません。その質問にはお答えできません。別の質問をしてください。"
    
    def add_rule(self, key, response):
        """新しいルールを追加"""
        self.rules[key] = response
        return f"新しいルールを追加しました: {key} → {response}"
    
    def show_rules(self):
        """現在のルールを表形式で表示"""
        rules_data = []
        for i, (key, response) in enumerate(self.rules.items()):
            rules_data.append([i+1, key, response])
        return rules_data


def add_new_rule(key, response):
    """新しいルールを追加"""
    if key and response:
        result = chatbot.add_rule(key, response)
        # 入力欄を空にするために空文字列を返す
        return result, "", "", chatbot.show_rules()
    return "キーワードと応答の両方を入力してください。", key, response, chatbot.show_rules()


# チャットボットのインスタンスを作成
chatbot = DictionaryChatbot()

# Gradioインターフェースを作成
with gr.Blocks(title="辞書型チャットボット", theme=gr.themes.Soft(primary_hue="blue", secondary_hue="gray")) as demo:
    gr.Markdown("# 辞書型チャットボット 🤖")
    
    with gr.Tab("チャット"):
        chatbot_interface = gr.Chatbot(
            label="Chatbot",
            height=400,
            value=[]
        )
        
        msg = gr.Textbox(
            label="質問",
            placeholder="質問を入力",
            lines=1,
            show_label=True
        )
        
        submit_btn = gr.Button("送信", variant="primary")
        
        def user_input(message, history):
            """ユーザー入力の処理"""
            try:
                # 入力の検証
                if not message.strip():
                    return "", history
                
                # 履歴を適切な形式に変換
                formatted_history = []
                if history and len(history) > 0:
                    for user_msg, ai_msg in history:
                        formatted_history.append([user_msg, ai_msg])
                
                # 応答を生成
                response = chatbot.find_response(message)
                
                # 履歴に追加
                if history is None:
                    history = []
                history.append([message, response])
                
                return "", history
                
            except Exception as e:
                error_msg = f"入力処理中にエラーが発生しました: {str(e)}"
                if history is None:
                    history = []
                history.append([message, error_msg])
                return "", history
    
    with gr.Tab("ルール管理"):
        gr.Markdown("## 現在のルール一覧")
        rules_display = gr.Dataframe(
            headers=["ID", "キーワード", "応答"],
            datatype=["number", "str", "str"],
            col_count=(3, "fixed"),
            value=chatbot.show_rules(),
            interactive=False
        )
        
        gr.Markdown("## 新しいルールを追加")
        with gr.Row():
            key_input = gr.Textbox(label="キーワード", placeholder="例：お疲れ様")
            response_input = gr.Textbox(label="応答", placeholder="例：お疲れ様です！")
        add_button = gr.Button("ルールを追加", variant="primary")
        add_result = gr.Textbox(label="結果", interactive=False)
        
        # イベントハンドラー
        add_button.click(
            fn=add_new_rule,
            inputs=[key_input, response_input],
            outputs=[add_result, key_input, response_input, rules_display]
        )
    
    with gr.Tab("使い方"):
        gr.Markdown("""
        ## 使い方
        
        ### チャット機能
        - キーワードと完全に一致する場合、対応する応答を返します
        
        ### ルール管理
        - 新しいルールを追加できます
        - 現在のルール一覧を確認できます
        
        ### 例
        - 「こんにちは」→「こんにちは！何かお手伝いできますか？」
        """)
        
    
    submit_btn.click(
        user_input,
        inputs=[msg, chatbot_interface],
        outputs=[msg, chatbot_interface]
    )
    
    msg.submit(
            user_input,
            inputs=[msg, chatbot_interface],
            outputs=[msg, chatbot_interface]
        )

# アプリケーションを起動
if __name__ == "__main__":
    # 利用可能なポートを探す
    port = find_available_port(7870)
    if port is None:
        print("エラー: 利用可能なポートが見つかりませんでした。")
        exit(1)
    
    print(f"ポート {port} で起動します...")
    
    try:
        demo.launch(
            inbrowser=True,
            server_name="0.0.0.0",
            server_port=port,
            share=False,
            show_error=True
        )
    except OSError as e:
        if "Address already in use" in str(e):
            print(f"ポート {port} が使用中です。別のポートを試します...")
            # 別のポートで再試行
            port = find_available_port(port + 1)
            if port is None:
                print("エラー: 利用可能なポートが見つかりませんでした。")
                exit(1)
            
            print(f"ポート {port} で再起動します...")
            demo.launch(
                inbrowser=True,
                server_name="0.0.0.0",
                server_port=port,
                share=False,
                show_error=True
            )
        else:
            print(f"起動中にエラーが発生しました: {e}")
            raise


D:\temp\ipykernel_17220\2855017450.py:69: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(


ポート 7873 で起動します...
* Running on local URL:  http://0.0.0.0:7873
* To create a public link, set `share=True` in `launch()`.


### 第2セル

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import socket

def find_available_port(start_port=7860, max_attempts=100):
    """利用可能なポートを見つける"""
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('0.0.0.0', port))
                return port
        except OSError:
            continue
    return None

class SarashinaChatbot:
    def __init__(self):
        self.model_name = "./sbintuitions/sarashina2.2-3B-instruct-v0.1"
        self.tokenizer = None
        self.model = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.load_model()
    
    def load_model(self):
        """モデルとトークナイザーをロード（4bit量子化）"""
        try:
            print(f"モデル {self.model_name} を4bit量子化でロード中...")
            
            # トークナイザーのロード
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            
            # pad_tokenの設定
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # 4bit量子化の設定
            from transformers import BitsAndBytesConfig
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True
            )
            
            # モデルのロード
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                quantization_config=quantization_config,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
            
            # モデル設定の更新
            if self.model.config.pad_token_id is None:
                self.model.config.pad_token_id = self.tokenizer.pad_token_id
            
            print(f"モデルのロードが完了しました。デバイス: {self.device} (4bit量子化)")
            
        except FileNotFoundError:
            print(f"エラー: モデルファイルが見つかりません: {self.model_name}")
            print("モデルパスが正しいか確認してください。")
        except ImportError as e:
            print(f"エラー: 必要なライブラリがインストールされていません: {e}")
        except Exception as e:
            print(f"モデルのロード中にエラーが発生しました: {e}")
            print("モデルファイルが正しいパスに存在することを確認してください。")
    
    def generate_response(self, message, history):
        """ユーザーのメッセージに対する応答を生成"""
        if self.model is None or self.tokenizer is None:
            return "モデルがロードされていません。モデルファイルを確認してください。"
        
        try:
            # 入力の検証
            if not message or not message.strip():
                return "メッセージが空です。何か入力してください。"
            
            # 会話履歴を含めたプロンプトの構築
            prompt = ""
            if history and len(history) > 0:
                for i, (user_msg, ai_msg) in enumerate(history):
                    prompt += f"質問{i+1}: {user_msg}\n回答{i+1}: {ai_msg}\n\n"
            
            prompt += f"質問: {message}\n回答: "
            
            # 入力のトークン化
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
            inputs = inputs.to(self.device)
            
            # 応答の生成
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=512,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id
                )
            
            # 応答のデコード
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # プロンプト部分を除去して応答のみを取得
            if prompt in response:
                response = response.replace(prompt, "").strip()
            
            # 空の応答の場合の処理
            if not response:
                return "応答を生成できませんでした。もう一度試してください。"
            
            return response
            
        except torch.cuda.OutOfMemoryError:
            return "GPUメモリが不足しています。モデルを再起動してください。"
        except Exception as e:
            return f"応答生成中にエラーが発生しました: {str(e)}"

def create_chatbot_interface():
    """Gradioインターフェースを作成"""
    chatbot = SarashinaChatbot()
    
    # Gradioインターフェースの定義
    with gr.Blocks(title="シンプルAI チャットボット", theme=gr.themes.Soft(primary_hue="blue", secondary_hue="gray")) as demo:
        gr.Markdown("# 🤖 シンプルAI チャットボット")
        gr.Markdown("sarashina2.2-3B-instruct-v0.1モデルを使用したチャットボットです。")
        gr.Markdown("会話の履歴はAI出力に反映されません")
        
        # チャット履歴
        chatbot_interface = gr.Chatbot(
            label="Chatbot",
            height=400,
            value=[]
        )
        
        # 入力フィールド
        msg = gr.Textbox(
            label="質問",
            placeholder="質問を入力してください",
            lines=1,
            show_label=True
        )
        
        # 送信ボタン
        submit_btn = gr.Button("送信", variant="primary")
        
        def user_input(message, history):
            """ユーザー入力の処理"""
            try:
                # 入力の検証
                if not message or not str(message).strip():
                    return "", history
                
                # 履歴を適切な形式に変換
                formatted_history = []
                if history and len(history) > 0:
                    for user_msg, ai_msg in history:
                        formatted_history.append([user_msg, ai_msg])
                
                # 応答を生成
                response = chatbot.generate_response(message, formatted_history)
                
                # 履歴に追加
                if history is None:
                    history = []
                history.append([message, response])
                
                return "", history
                
            except Exception as e:
                error_msg = f"入力処理中にエラーが発生しました: {str(e)}"
                if history is None:
                    history = []
                history.append([message, error_msg])
                return "", history
        
        # イベントハンドラーの設定
        submit_btn.click(
            user_input,
            inputs=[msg, chatbot_interface],
            outputs=[msg, chatbot_interface]
        )
        
        # エンターキーで送信
        msg.submit(
            user_input,
            inputs=[msg, chatbot_interface],
            outputs=[msg, chatbot_interface]
        )
    
    return demo

if __name__ == "__main__":
    # チャットボットインターフェースを作成して起動
    demo = create_chatbot_interface()
    
    # 利用可能なポートを探す（7860から10件確認）
    port = find_available_port(7860, 10)
    if port is None:
        print("エラー: ポート7860-7869で利用可能なポートが見つかりませんでした。")
        print("他のアプリケーションを終了してから再試行してください。")
        exit(1)
    
    print(f"ポート {port} で起動します...")
    
    try:
        demo.launch(
            inbrowser=True,
            server_name="127.0.0.1",
            server_port=port,
            share=False,
            show_error=True
        )
    except Exception as e:
        print(f"起動中にエラーが発生しました: {e}")
        raise


モデル ./sbintuitions/sarashina2.2-3B-instruct-v0.1 を4bit量子化でロード中...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

D:\temp\ipykernel_17220\836626979.py:132: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_interface = gr.Chatbot(


モデルのロードが完了しました。デバイス: cuda (4bit量子化)
ポート 7860 で起動します...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
